In [1]:
%matplotlib qt

In [37]:
from renewableopt.initial_data_wrangling import load_geojson
from shapely import Point, LineString
import geopandas as gp
import contextily as cx

In [3]:
PROJ_M = 3857  # Mercator projection in meters
PROJ_GEO = 4326  # Geodetic projection
m_per_mile = 1609.34  # meters/mile

In [4]:
geo = load_geojson()
geo_m = geo.to_crs(PROJ_M)

68
Index(['OBJECTID', 'Shape_Length', 'Shape_Area', 'Name', 'CODE', 'CREATEDBY',
       'CREATEDDATE', 'UPDATEDBY', 'DOCYR', 'DOCNO', 'DOCDATE', 'SUBJECT',
       'UPDATEDATE', 'GlobalID', 'geometry'],
      dtype='object')


In [5]:
geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
geo_m.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [25]:
buffered = geo_m.buffer(50 * m_per_mile).to_crs(4326)
boundary = gp.GeoSeries(buffered.unary_union, crs=PROJ_GEO).boundary

In [46]:
transmission_line = gp.GeoSeries(LineString([
    (-115.59711027151646, 33.15575764142154),
    (-115.58043553007757, 32.81298135361883),
    (-116.37694431023587, 32.61308515918179),
#     (-116.129125, 32.619179),
    (-116.94686005756834, 32.7517890818103)
]), crs=PROJ_GEO)

In [43]:
geothermal_plants = gp.GeoSeries([
#     Point(-115.61667, 33.1633)  # Salton Sea Geothermal
    Point(-115.647567, 33.157561),
    Point(-115.639084, 33.156842),
    Point(-115.643986, 33.159869),
    Point(-115.639062, 33.155532), 
    Point(-115.638800, 33.152789), 
    Point(-115.616119, 33.164385), 
    Point(-115.619542, 33.162710),
    Point(-115.604832, 33.177273), 
    Point(-115.565306, 33.177459), 
], crs=PROJ_GEO)

solar_plants = gp.GeoSeries([
    Point(-116.129125, 32.619179),
    Point(-116.001926, 33.10889),
    Point(-116.032298, 33.119983)
], crs=PROJ_GEO)


In [44]:
boundary

0    LINESTRING (-116.85560 31.94281, -116.86851 31...
dtype: geometry

In [47]:
base = geo.to_crs(PROJ_M).plot(alpha=0.5)
base = boundary.to_crs(PROJ_M).plot(ax=base)
cx.add_basemap(base)
geothermal_plants.to_crs(PROJ_M).plot(color='orange', ax=base, label="Geothermal")
solar_plants.to_crs(PROJ_M).plot(color='red', ax=base, label="Solar/Battery Hybrid")
transmission_line.to_crs(PROJ_M).plot(color='yellow', ax=base, label="Transmission")
base.axis("off")
base.legend()